In [7]:
# Importar AIF360
from aif360.datasets import AdultDataset
from aif360.algorithms.preprocessing import Reweighing
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric

import matplotlib.pyplot as plt
%matplotlib inline

print("¡Todo importado correctamente!")

¡Todo importado correctamente!


/usr/local/lib/python3.12/dist-packages/inFairness/utils/ndcg.py:37: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  vect_normalized_discounted_cumulative_gain = vmap(
/usr/local/lib/python3.12/dist-packages/inFairness/utils/ndcg.py:48: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  monte_carlo_vect_ndcg = vmap(vect_normalized_discounted

In [9]:
import os

# Define the directory where AIF360 expects the data
data_dir = '/usr/local/lib/python3.12/dist-packages/aif360/data/raw/adult'

# Create the directory if it doesn't exist
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    print(f"Directory created: {data_dir}")

# List of files to download
files_to_download = [
    'adult.data',
    'adult.test',
    'adult.names'
]

# Base URL for the files
base_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/'

# Download files if they don't exist in the data_dir
for file_name in files_to_download:
    file_path = os.path.join(data_dir, file_name)
    if not os.path.exists(file_path):
        print(f"Downloading {file_name}...")
        !wget -P {data_dir} {base_url}{file_name}
    else:
        print(f"{file_name} already exists.")

# Cargar el dataset Adult directamente (AIF360 lo descarga automático la primera vez)
dataset_orig = AdultDataset(
    protected_attribute_names=['sex'],  # Atributo protegido: género
    privileged_classes=[['Male']],      # Grupo privilegiado: hombres
    features_to_drop=['race']           # Opcional: quitamos raza por ahora para enfocarnos en género
)

# Ver información básica
print("Dataset cargado correctamente")
print(f"Número de muestras: {dataset_orig.features.shape[0]}")
print(f"Número de características: {dataset_orig.features.shape[1]}")
print("Etiquetas (income >50K):", dataset_orig.label_names)
print("Grupos protegidos:", dataset_orig.protected_attribute_names)

--2025-12-28 18:41:05--  https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘/usr/local/lib/python3.12/dist-packages/aif360/data/raw/adult/adult.data’

adult.data              [   <=>              ]   3.79M  7.97MB/s    in 0.5s    

2025-12-28 18:41:06 (7.97 MB/s) - ‘/usr/local/lib/python3.12/dist-packages/aif360/data/raw/adult/adult.data’ saved [3974305]

--2025-12-28 18:41:06--  https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘/usr/local/lib/python3.12/dist-packages/

Dataset cargado correctamente
Número de muestras: 45222
Número de características: 98
Etiquetas (income >50K): ['income-per-year']
Grupos protegidos: ['sex']


In [10]:
# Convertir a pandas para ver las primeras filas
df, _ = dataset_orig.convert_to_dataframe()
df.head(10)

,age,fnlwgt,education-num,sex,capital-gain,capital-loss,hours-per-week,workclass=Federal-gov,workclass=Local-gov,workclass=Private,...,native-country=Puerto-Rico,native-country=Scotland,native-country=South,native-country=Taiwan,native-country=Thailand,native-country=Trinadad&Tobago,native-country=United-States,native-country=Vietnam,native-country=Yugoslavia,income-per-year
0,25.0,226802.0,7.0,1.0,0.0,0.0,40.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,38.0,89814.0,9.0,1.0,0.0,0.0,50.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,28.0,336951.0,12.0,1.0,0.0,0.0,40.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,44.0,160323.0,10.0,1.0,7688.0,0.0,40.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
5,34.0,198693.0,6.0,1.0,0.0,0.0,30.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
7,63.0,104626.0,15.0,1.0,3103.0,0.0,32.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
8,24.0,369667.0,10.0,0.0,0.0,0.0,40.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9,55.0,104996.0,4.0,1.0,0.0,0.0,10.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
10,65.0,184454.0,9.0,1.0,6418.0,0.0,40.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
11,36.0,212465.0,13.0,1.0,0.0,0.0,40.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [12]:
# Importar clases
from aif360.datasets import AdultDataset
from aif360.metrics import BinaryLabelDatasetMetric

# Cargar dataset SOLO con género como atributo protegido y quitar 'race' para evitar el error
dataset_orig = AdultDataset(
    protected_attribute_names=['sex'],
    privileged_classes=[['Male']],
    features_to_drop=['race']  # Esto elimina la columna problemática 'race'
)

# Grupos
privileged_groups = [{'sex': 1.0}]
unprivileged_groups = [{'sex': 0.0}]

# Métricas de bias
metric_orig = BinaryLabelDatasetMetric(
    dataset_orig,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

# Resultados
print("=== Detección de Bias por Género (Adult Dataset) ===")
print(f"Tasa ingresos >50K hombres: {metric_orig.base_rate(privileged=True):.1%}")
print(f"Tasa ingresos >50K mujeres: {metric_orig.base_rate(privileged=False):.1%}")
print(f"Disparate Impact: {metric_orig.disparate_impact():.3f}")
print(f"Diferencia media: {metric_orig.mean_difference():.3f}")

if metric_orig.disparate_impact() < 0.8:
    print("\n⚠️ Bias significativo contra mujeres")
else:
    print("\n✅ Sin bias grave")

=== Detección de Bias por Género (Adult Dataset) ===
Tasa ingresos >50K hombres: 31.2%
Tasa ingresos >50K mujeres: 11.4%
Disparate Impact: 0.363
Diferencia media: -0.199

⚠️ Bias significativo contra mujeres


In [13]:
# Importar el algoritmo de mitigación
from aif360.algorithms.preprocessing import Reweighing

# Aplicar Reweighing
RW = Reweighing(
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

# Transformar el dataset (crea una versión rebalanceada)
dataset_transf = RW.fit_transform(dataset_orig)

# Calcular métricas en el dataset mitigado
metric_transf = BinaryLabelDatasetMetric(
    dataset_transf,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

# Mostrar comparación ANTES vs DESPUÉS
print("=== Comparación Antes vs Después de Mitigación (Reweighing) ===")
print("\nANTES (dataset original):")
print(f"  Disparate Impact: {metric_orig.disparate_impact():.3f}")
print(f"  Diferencia media: {metric_orig.mean_difference():.3f}")

print("\nDESPUÉS (dataset rebalanceado):")
print(f"  Disparate Impact: {metric_transf.disparate_impact():.3f}")
print(f"  Diferencia media: {metric_transf.mean_difference():.3f}")

# Interpretación final
if abs(metric_transf.mean_difference()) < 0.01 and metric_transf.disparate_impact() > 0.99:
    print("\n✅ Bias prácticamente eliminado (Statistical Parity alcanzado)")
else:
    print("\n✓ Bias reducido significativamente")

=== Comparación Antes vs Después de Mitigación (Reweighing) ===

ANTES (dataset original):
  Disparate Impact: 0.363
  Diferencia media: -0.199

DESPUÉS (dataset rebalanceado):
  Disparate Impact: 1.000
  Diferencia media: 0.000

✅ Bias prácticamente eliminado (Statistical Parity alcanzado)
